<a href="https://colab.research.google.com/github/sunbui/breast-cancer-analytics/blob/main/biopython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    import google.colab
    # Running on Google Colab, so install Biopython first
    !pip install biopython
except ImportError:
    pass

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 11.3 MB/s 


In [2]:
import os
import sys
from urllib.request import urlretrieve
import Bio
from Bio import SeqIO, SearchIO, Entrez
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio.Blast import NCBIWWW
from Bio.Data import CodonTable

print("Python version:", sys.version_info)
print("Biopython version:", Bio.__version__)

Python version: sys.version_info(major=3, minor=7, micro=15, releaselevel='final', serial=0)
Biopython version: 1.79


In [3]:
import pymongo

input file 

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving example.fasta to example.fasta


# Chap3: Sequence objects

We are created a simple protein sequence: AGCT 

Each letter represents Alanine, Glycine, Cysteine and Threonine
Each Seq object has two important attributes − data and alphabet 

Since Sep 2020, BioPython has removed Bio.Alphabet module in 1.79 version. 



3.1 Sequences act like strings

In [ ]:

my_seq = Seq("GATCG")
for index, letter in enumerate(my_seq):
  print("%i %s" % (index, letter))






0 G
1 A
2 T
3 C
4 G


In [ ]:
print(len(my_seq))
print(my_seq[0]) #first letter
print(my_seq[2]) #third letter
print(my_seq[-1]) #last letter

5
G
T
G


The Seq object has a .count() method, just like a string. Note that this means that like a Python
string, this gives a non-overlapping count:

In [ ]:

"AAAA".count("AA")
#same with 
Seq("AAAA").count("AA")

2

For some biological uses, you may actually want an overlapping count. When searching for single letters, this makes no difference

In [ ]:

my_seq2 = Seq("GATCGATGGGCCTATATAGGATCGAAAATCGC")
len(my_seq2)
my_seq2.count("G")
100 * float(my_seq2.count("G") + my_seq2.count("C")) / len(my_seq2)

46.875

While you could use the above snippet of code to calculate a GC%, note that the Bio.SeqUtils module has several GC functions already built. 

In [ ]:

my_seq = Seq("GATCGATGGGCCTATATAGGATCGAAAATCGC")
GC(my_seq)


46.875

3.2 Slicing a sequence

In [ ]:
print(my_seq[::-1]) #  reverse
print(my_seq.reverse_complement())
print(my_seq.reverse_complement_rna())
my_seq[0::3]

CGCTAAAAGCTAGGATATATCCGGGTAGCTAG
GCGATTTTCGATCCTATATAGGCCCATCGATC
GCGAUUUUCGAUCCUAUAUAGGCCCAUCGAUC


Seq('GCTGTAGTAAG')

3.3 Turning Seq objects into strings

In [ ]:
str(my_seq)

'GATCGATGGGCCTATATAGGATCGAAAATCGC'

In [ ]:
fasta_format_string = ">Name\n%s\n" % my_seq
print(fasta_format_string)

>Name
GATCGATGGGCCTATATAGGATCGAAAATCGC



3.4 Concatenating or adding sequences

In [ ]:
protein_seq = Seq("EVRNAK")
dna_seq = Seq("ACGT")
protein_seq + dna_seq #add any two Seq objects
list_of_seqs = [Seq("ACGT"), Seq("AACC"), Seq("GGTT")]
concatenated = Seq("")
#add any three Seq objects
for s in list_of_seqs:
  concatenated += s
concatenated

Seq('ACGTAACCGGTT')

Like Python strings, Biopython Seq also has a .join method:

In [ ]:
contigs = [Seq("ATG"), Seq("ATCCCG"), Seq("TTGCA")]
spacer = Seq("N"*10)
spacer.join(contigs)

Seq('ATGNNNNNNNNNNATCCCGNNNNNNNNNNTTGCA')

3.5 Changing case

In [ ]:
dna_seq = Seq("acgtACGT")
print(dna_seq)
print(dna_seq.upper())
print(dna_seq.lower())
#These are useful for doing case insensitive matching:
print("GTAC" in dna_seq)
"GTAC" in dna_seq.upper()

acgtACGT
ACGTACGT
acgtacgt
False


True

3.6 Nucleotide sequences and (reverse) complements


In [ ]:
my_seq = Seq("GATCGATGGGCCTATATAGGATCGAAAATCGC")
print(my_seq)
print(my_seq.complement())
print(my_seq.reverse_complement())
print(my_seq.reverse_complement_rna())
print(my_seq[::-1])


GATCGATGGGCCTATATAGGATCGAAAATCGC
CTAGCTACCCGGATATATCCTAGCTTTTAGCG
GCGATTTTCGATCCTATATAGGCCCATCGATC
CGCTAAAAGCTAGGATATATCCGGGTAGCTAG
GCGAUUUUCGAUCCUAUAUAGGCCCAUCGAUC


If you do accidentally end up trying to do something weird like taking the (reverse)complement of a
protein sequence, the results are biologically meaningless,
Here the letter “E” is not a valid IUPAC ambiguity code for nucleotides, so was not complemented.
However, “V” means “A”, “C” or “G” and has complement “B“, and so on.

In [ ]:
protein_seq = Seq("EVRNAK")
protein_seq.complement()

Seq('EBYNTM')

3.7 Transcription
TCAG => CUGA to give the mRNA
we can get the mRNA sequence just by switching
T → U.

In [ ]:
coding_dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG")
template_dna = coding_dna.reverse_complement()
messenger_rna = coding_dna.transcribe()
print(coding_dna)
print(template_dna)
print(messenger_rna)
#As you can see, all this does is to replace T by U.
#If you do want to do a true biological transcription starting with the template strand, 
#then this becomes a two-step process:


ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG
CTATCGGGCACCCTTTCAGCGGCCCATTACAATGGCCAT
AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG
AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG
ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG


In [ ]:
#The Seq object also includes a back-transcription method for going from the mRNA to the coding strand
# of the DNA. Again, this is a simple U → T substitution:
print(template_dna.reverse_complement().transcribe())
print(messenger_rna.back_transcribe())


AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG
ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG


3.8 Translation

Sticking with the same example discussed in the transcription section above, now let’s translate this mRNA
into the corresponding protein sequence - again taking advantage of one of the Seq object’s biological methods:

In [ ]:
messenger_rna = Seq("AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG")
print(messenger_rna)
print(messenger_rna.translate())

AUGGCCAUUGUAAUGGGCCGCUGAAAGGGUGCCCGAUAG
MAIVMGR*KGAR*


In [ ]:
#You can also translate directly from the coding strand DNA sequence: it gives you the same result
coding_dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG")
print(coding_dna)
print(coding_dna.translate())

ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG
MAIVMGR*KGAR*


NCBI

You can also specify the table using the NCBI table number which is shorter, and often included in the
feature annotation of GenBank files:

In [ ]:
print(coding_dna.translate(table="Vertebrate Mitochondrial"))
print(coding_dna.translate(table=2)) #from NCBI Genebank 
print(coding_dna.translate(to_stop=True))
print(coding_dna.translate(table=2, to_stop=True))

MAIVMGRWKGAR*
MAIVMGRWKGAR*
MAIVMGR
MAIVMGRWKGAR


In [ ]:
gene = Seq("GTGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGGTCGCTCCCATGGCA"
"GCACAGGCTGCGGAAATTACGTTAGTCCCGTCAGTAAAATTACAGATAGGCGATCGTGAT"
"AATCGTGGCTATTACTGGGATGGAGGTCACTGGCGCGACCACGGCTGGTGGAAACAACAT"
"TATGAATGGCGAGGCAATCGCTGGCACCTACACGGACCGCCGCCACCGCCGCGCCACCAT"
"AAGAAAGCTCCTCATGATCATCACGGCGGTCATGGTCCAGGCAAACATCACCGCTAA")
print(gene.translate(table="Bacterial"))
print(gene.translate(table="Bacterial", to_stop=True))
print(gene.translate(table="Bacterial", cds=True))

VKKMQSIVLALSLVLVAPMAAQAAEITLVPSVKLQIGDRDNRGYYWDGGHWRDHGWWKQHYEWRGNRWHLHGPPPPPRHHKKAPHDHHGGHGPGKHHR*
VKKMQSIVLALSLVLVAPMAAQAAEITLVPSVKLQIGDRDNRGYYWDGGHWRDHGWWKQHYEWRGNRWHLHGPPPPPRHHKKAPHDHHGGHGPGKHHR
MKKMQSIVLALSLVLVAPMAAQAAEITLVPSVKLQIGDRDNRGYYWDGGHWRDHGWWKQHYEWRGNRWHLHGPPPPPRHHKKAPHDHHGGHGPGKHHR


3.9 Translation Tables: 

In [ ]:
standard_table = CodonTable.unambiguous_dna_by_name["Standard"]
mito_table = CodonTable.unambiguous_dna_by_name["Vertebrate Mitochondrial"]
print(standard_table)
print(mito_table)
print(mito_table.stop_codons)
print(mito_table.forward_table["ACG"])

Table 1 Standard, SGC0

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L   | TCA S   | TAA Stop| TGA Stop| A
T | TTG L(s)| TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L(s)| CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I   | ACT T   | AAT N   | AGT S   | T
A | ATC I   | ACC T   | AAC N   | AGC S   | C
A | ATA I   | ACA T   | AAA K   | AGA R   | A
A | ATG M(s)| ACG T   | AAG K   | AGG R   | G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   | GCA A   | GAA E   | GGA G   | A
G | GTG V   | GCG A   | GAG E   | GGG G   | G
--+---------

In [ ]:
table = CodonTable.unambiguous_dna_by_name["Standard"] 
print(table)

Table 1 Standard, SGC0

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L   | TCA S   | TAA Stop| TGA Stop| A
T | TTG L(s)| TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L(s)| CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I   | ACT T   | AAT N   | AGT S   | T
A | ATC I   | ACC T   | AAC N   | AGC S   | C
A | ATA I   | ACA T   | AAA K   | AGA R   | A
A | ATG M(s)| ACG T   | AAG K   | AGG R   | G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   | GCA A   | GAA E   | GGA G   | A
G | GTG V   | GCG A   | GAG E   | GGG G   | G
--+---------

3.10 Comparing Seq objects

In [ ]:
seq1 = Seq("ACGT")
'ACGT' == seq1

True

3.11 Sequences with unknown sequence contents

In [ ]:
unknown_seq = Seq(None, 10)
print(len(unknown_seq))

10


3.12 MutableSeq objects 

In [ ]:
#MutableSeq objects = unchangable 
my_seq = Seq("GCCATTGTAATGGGCCGCTGAAAGGGTGCCCGA") 
# you can't print my_seq[5] = 'c' because Seq object act like a string
# however, you can convert it into a mutable sequence (a MutableSeq object)
from Bio.Seq import MutableSeq
mutable_seq =  MutableSeq(my_seq)
mutable_seq[5] ='C'
mutable_seq2 = MutableSeq("GCCATTGTAATGGGCCGCTGAAAGGGTGCCCGA")
print(mutable_seq)
# unlike Seq object, the mutableSeq objects methods beloew act like onsite (khong thay doi)
mutable_seq2.reverse()
mutable_seq2.reverse_complement()


GCCATCGTAATGGGCCGCTGAAAGGGTGCCCGA


Do note that unlike the Seq object, the MutableSeq object’s methods like reverse_complement() and
reverse() act in-situ!
An important technical difference between mutable and immutable objects in Python means that you
can’t use a MutableSeq object as a dictionary key, but you can use a Python string or a Seq object in this
way.
Once you have finished editing your a MutableSeq object, it’s easy to get back to a read-only Seq object
should you need to:

In [ ]:
new_seq = Seq(mutable_seq)
new_seq

Seq('GCCATCGTAATGGGCCGCTGAAAGGGTGCCCGA')

3.13 UnknownSeq objects
The UnknownSeq object is a subclass of the basic Seq object and its purpose is to represent a sequence where we know the length, but not the actual letters making it up. You could of course use a normal Seq object in this situation, but it wastes rather a lot of memory to hold a string of a million “N” characters when you could just store a single letter “N” and the desired length as an integer.

In [ ]:
from Bio.Seq import UnknownSeq
unk = UnknownSeq(20)
# print(unk)
unk_dna = UnknownSeq(20, character="N")
print(unk_dna)
unk_dna.complement()


NNNNNNNNNNNNNNNNNNNN


/usr/local/lib/python3.7/dist-packages/Bio/Seq.py:2010: BiopythonDeprecationWarning: UnknownSeq(length) is deprecated; please use Seq(None, length) instead.
  BiopythonDeprecationWarning,


UnknownSeq(20, character='N')

In [ ]:
unk_dna.reverse_complement()

/usr/local/lib/python3.7/dist-packages/Bio/Seq.py:2010: BiopythonDeprecationWarning: UnknownSeq(length) is deprecated; please use Seq(None, length) instead.
  BiopythonDeprecationWarning,


UnknownSeq(20, character='N')

In [ ]:
unk_dna.transcribe()

/usr/local/lib/python3.7/dist-packages/Bio/Seq.py:2010: BiopythonDeprecationWarning: UnknownSeq(length) is deprecated; please use Seq(None, length) instead.
  BiopythonDeprecationWarning,


UnknownSeq(20, character='N')

In [ ]:
unk_protein = unk_dna.translate()
unk_protein


/usr/local/lib/python3.7/dist-packages/Bio/Seq.py:2010: BiopythonDeprecationWarning: UnknownSeq(length) is deprecated; please use Seq(None, length) instead.
  BiopythonDeprecationWarning,


UnknownSeq(6, character='X')

In [ ]:
len(unk_protein)

6

3.14 Working with strings directly

In [ ]:
from Bio.Seq import reverse_complement, transcribe, back_transcribe, translate
my_string = "GCTGTTATGGGTCGTTGGAAGGGTGGTCGTGCTGCTGGTTAG"
print(reverse_complement(my_string))
print(transcribe(my_string))
print(back_transcribe(my_string))
print(translate(my_string))

CTAACCAGCAGCACGACCACCCTTCCAACGACCCATAACAGC
GCUGUUAUGGGUCGUUGGAAGGGUGGUCGUGCUGCUGGUUAG
GCTGTTATGGGTCGTTGGAAGGGTGGTCGTGCTGCTGGTTAG
AVMGRWKGGRAAG*


Chapter 4- Sequence annotation objects => for GenBank or EMBL files not for FASTA files 


# 4.1 The SeqRecord object
The SeqRecord class itself is quite simple, and offers the following information as attributes:

**.seq** – The sequence itself, typically a Seq object.

**.id** – The primary ID used to identify the sequence – a string. In most cases this is something like an
accession number.

**.name** – A “common” name/id for the sequence – a string. In some cases this will be the same as the
accession number, but it could also be a clone name. I think of this as being analogous to the LOCUS
id in a GenBank record.

**.description** – A human readable description or expressive name for the sequence – a string.

**.letter_annotations** – Holds per-letter-annotations using a (restricted) dictionary of additional information
about the letters in the sequence. The keys are the name of the information, and the information is
contained in the value as a Python sequence (i.e. a list, tuple or string) with the same length as
the sequence itself. This is often used for quality scores (e.g. Section 20.1.6) or secondary structure
information (e.g. from Stockholm/PFAM alignment files).

**.annotations** – A dictionary of additional information about the sequence. The keys are the name of
the information, and the information is contained in the value. This allows the addition of more
“unstructured” information to the sequence.

**.features** – A list of SeqFeature objects with more structured information about the features on a sequence
(e.g. position of genes on a genome, or domains on a protein sequence). The structure of sequence
features is described below in Section 4.3.

**.dbxrefs** - A list of database cross-references as strings.


In [ ]:
from Bio.SeqRecord import SeqRecord


4.2 Creating a SeqRecord
4.2.1 SeqRecord objects from scratch 